In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [ ]:
#Cambiar la estrucutra para las ids, tengo que usar las url para las series y con eso sacar los map id 

In [75]:
def convert_k(valor):
    "remove k in money columns and change the format to int"
    if 'k' in valor:
        return int(float(valor.replace('k', '')) * 1000)

def find_files_by_prefix(root_folder, prefix):
    matched_files = []
    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.startswith(prefix):
                full_path = os.path.join(dirpath, file)
                matched_files.append(full_path)
    return matched_files

def concat_from_list(file_list,encoding='iso-8859-1'):
    dataframes = []
    for file in file_list:
        try:
            df = pd.read_csv(file,encoding=encoding)
            if not df.empty:
                dataframes.append(df)
            else:
                print(f"empty file: {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        print("Load file fail")
        return pd.DataFrame()

def concat_csv_from_different_folders(folder="csv",prefix=None):
    if prefix is None:
        print("Add a prefix")

    file_list = find_files_by_prefix(root_folder=folder,prefix=prefix)
    df_concat = concat_from_list(file_list)
    return df_concat

def get_game_instance(value):
    last_char = value.split("-")[-1]
    return last_char

def text_to_index(df,name,number=0,extra_id=""):
    name_id = name+'_id'
    df[name_id] = df.index + number
    df[name_id] = name + "_" + df[name_id].astype(str) + extra_id
    return df

In [111]:
def tournament_names(folder='csv'):
    tournament_list = []
    for name in os.listdir(folder):
        path = os.path.join(folder, name)
        if os.path.isdir(path):
            tournament_list.append(name)
    return tournament_list

def region_by_id(touranment_name, region):
    for _, row in region.iterrows():
        if row['region'].lower() in touranment_name.lower():
            return row['reg_id']
    return "reg_4"

def create_draft_table(df):
    filas_transformadas = []

    for index, row in df.iterrows():
        picks = [
            row['team_1_select_1'], row['team_2_select_1'],
            row['team_1_select_2'], row['team_2_select_2'],
            row['team_1_select_3'], row['team_2_select_3'],
            row['decider']
        ]

        for pick_num, map_name in enumerate(picks, start=1):
            filas_transformadas.append({
                'team': row['team'],
                'series_id': row['series_id'],
                
                'order': row['order'],
                'bo': row['bo'],
                'pick': pick_num,
                'map_name': map_name,
                "match_instance" : row["match_instance"],
                "vlr_id":row["vlr_id"],
                'series_id':row['series_id'],
                'reg_id':row['reg_id'],
                'tour_id':row['tour_id']
            })

    new_df = pd.DataFrame(filas_transformadas)
    return new_df

def first_ban(row):
    if (row['match_instance'] != "gf"):
        if (row['pick'] == 1):        
            return 1
        else:
            return 0
    elif (row['match_instance'] == "gf"):
        if (row['pick'] == 1 or row['pick'] == 2):        
            return 1
        else:
            return 0
    
    else:
         return 0

def second_ban(row):
    
        if row['pick'] == 5 and row['bo'] == 3:
            return 1
        else:
            return 0

def first_pick(row):

        if row['pick'] == 3:
            return 1
        else:
            return 0
        
   
def second_pick(row):
        if row['pick'] == 5 and row['bo'] == 5:
            return 1
        else:
            return 0
        
def decider_pick(row):
    if row['pick'] == 7:
        return 0.5
    else:
        return 0

In [5]:
#This should round only once.
#This should be create in another way, probably from a json 
region_name = ["americas","emea","china","pacific","global"]

df_region = pd.DataFrame(data=region_name, columns=["region"])
text_to_index(df_region,"reg",number=0,extra_id="")

#This should run each time a new tournament is add

tournaments = tournament_names('csv')
df_tournaments = pd.DataFrame(data=tournaments,columns=["tournament_name"])
tournament_name_list = ['Valorant Masters Toronto 2025', 'VCT 2025: Americas Stage 1',
       'VCT 2025: China Stage 1', 'VCT 2025: EMEA Stage 1',
       'VCT 2025: Pacific Stage 1']
df_tournaments["event"] = pd.Series(tournament_name_list)

df_tournaments['reg_id'] = df_tournaments['event'].apply(lambda x: region_by_id(x, df_region))

text_to_index(df_tournaments,"tour")

df_region.to_csv(path_or_buf='tables/table_region.csv',index=False,encoding='iso-8859-1')
df_tournaments.to_csv(path_or_buf='tables/table_tournament.csv',index=False,encoding='iso-8859-1')

In [10]:
df_draft_concat = concat_csv_from_different_folders(folder="csv", prefix="draft_")
df_team = pd.DataFrame(df_draft_concat["team"].unique(), columns=["team"])

text_to_index(df_team,"team")

df_team.to_csv(path_or_buf='tables/table_teams.csv',index=False,encoding='iso-8859-1')

In [ ]:
# #CREO QUE NO LO VOY A USAR 

# #teams by region:
# df_teams_china = pd.read_csv("csv/vct_2025_china_stage_1/draft_vct_2025_china_stage_1.csv",encoding='iso-8859-1')
# df_teams_americas = pd.read_csv("csv/vct_2025_americas_stage_1/draft_vct_2025_americas_stage_1.csv",encoding='iso-8859-1')
# df_teams_emea = pd.read_csv("csv/vct_2025_emea_stage_1/draft_vct_2025_emea_stage_1.csv",encoding='iso-8859-1')
# df_teams_pacific = pd.read_csv("csv/vct_2025_pacific_stage_1/draft_vct_2025_pacific_stage_1.csv",encoding='iso-8859-1')

# #this should run one by year
# df_teams_amer_for_merge = pd.DataFrame(df_teams_americas["team"].unique(),columns=["team"])
# df_teams_amer_for_merge["team_reg"] = "reg_0"

# df_teams_emea_for_merge = pd.DataFrame(df_teams_emea["team"].unique(),columns=["team"])
# df_teams_emea_for_merge["team_reg"] = "reg_1"

# df_teams_china_for_merge = pd.DataFrame(df_teams_china["team"].unique(),columns=["team"])
# df_teams_china_for_merge["team_reg"] = "reg_2"

# df_teams_apac_for_merge = pd.DataFrame(df_teams_pacific["team"].unique(),columns=["team"])
# df_teams_apac_for_merge["team_reg"] = "reg_3"

# df_teams_concat = pd.concat(
#     [
#         df_teams_amer_for_merge,
#         df_teams_emea_for_merge,
#         df_teams_china_for_merge,
#         df_teams_apac_for_merge,
#     ],
#     ignore_index=True,
# )

# text_to_index(df_teams_concat,"team")
 

In [11]:
#Update when a new player is add
df_players_stats = concat_csv_from_different_folders(folder="csv", prefix="player_stats")
df_players = df_players_stats[['player','team']].drop_duplicates(subset=['player'],ignore_index=True)
text_to_index(df_players,"player")

df_players_id = pd.merge(df_players, df_team[["team","team_id"]],how="left", left_on="team", right_on="team")

#Le saco las regiones a los players ? 
#df_players_id = pd.merge(df_players_id, df_teams_concat[["team_id","region"]],how="left", left_on="team_id", right_on="team_id")

df_players_id.to_csv(path_or_buf='tables/table_players.csv',index=False,encoding='iso-8859-1')

In [ ]:
def match_id_vlr(url):
    match = re.search(r"vlr\.gg/(\d+)", url)
    if match:
        vlr_id = match.group(1)
        return vlr_id


df_round_detail = concat_csv_from_different_folders(folder="csv", prefix="round_detail")

df_filter_map = df_round_detail["map"] != "all"
df_round_detail_filter = df_round_detail[df_filter_map]
df_round_detail_filter["vlr_id"] = df_round_detail_filter["source_url"].apply(
    match_id_vlr
)
df_round_detail_filter["vlr_id-map"] = (
    df_round_detail_filter["vlr_id"] + "-" + df_round_detail_filter["map"]
)

df_maps_id = pd.DataFrame(
    df_round_detail_filter["vlr_id-map"].unique(), columns=["vlr_id-map"]
)
text_to_index(df_maps_id, "map")

df_maps_id[["vlr_id", "map"]] = df_maps_id["vlr_id-map"].str.rsplit(
    "-", n=1, expand=True
)

df_match_id = pd.DataFrame(data=df_maps_id["vlr_id"].unique(), columns=["vlr_id"])
text_to_index(df_match_id, "series")

df_round_detail_filter = pd.merge(
    df_round_detail_filter,
    df_tournaments[["event", "reg_id", "tour_id"]],
    how="left",
    left_on="event",
    right_on="event",
)

df_match_id = pd.merge(
    df_match_id,
    df_round_detail_filter[["vlr_id", "reg_id", "tour_id"]],
    how="left",
    left_on="vlr_id",
    right_on="vlr_id",
)

df_match_id.drop_duplicates(inplace=True, ignore_index=True)

df_maps_id = pd.merge(
    df_maps_id,
    df_match_id,
    how="left",
    left_on="vlr_id",
    right_on="vlr_id",
)

df_tournament_played = df_round_detail_filter[["teamA","event","reg_id","tour_id"]].copy()
df_tournament_played.drop_duplicates(inplace=True, ignore_index=True)

df_maps_id.to_csv(path_or_buf='tables/table_maps_id.csv',index=False,encoding='iso-8859-1')

df_match_id.to_csv(path_or_buf='tables/table_match_id.csv',index=False,encoding='iso-8859-1')

#df_round_detail_filter.to_csv(path_or_buf='tables/table_round_detail.csv',index=False,encoding='iso-8859-1') Esto hay que limpiar

df_tournament_played.to_csv(path_or_buf='tables/table_tournament_played.csv',index=False,encoding='iso-8859-1')


In [ ]:
#Player performance
df_player_performance = concat_csv_from_different_folders(folder="csv",prefix="player_performance")

df_filter_map = df_player_performance["map"] != "all"
df_player_performance = df_player_performance[df_filter_map]
df_player_performance["vlr_id"] = df_player_performance["source_url"].apply(
    match_id_vlr
)
df_player_performance["vlr_id-map"] = (
    df_player_performance["vlr_id"] + "-" + df_player_performance["map"]
)

df_player_performance = pd.merge(df_player_performance, df_maps_id[["vlr_id-map","map_id","series_id","reg_id","tour_id"]],how="left", left_on="vlr_id-map", right_on="vlr_id-map")

df_player_performance.to_csv(path_or_buf='tables/table_player_performance.csv',index=False,encoding='iso-8859-1')

In [ ]:
#Team economy stats 

df_team_economy = concat_csv_from_different_folders(folder="csv", prefix="team_economy")

columns_update = ["team_a_economy","team_b_economy","team_a_bank","team_b_bank"]

for column_name in columns_update:
    df_team_economy[column_name] = df_team_economy[column_name].apply(convert_k)

df_team_economy["vlr_id"] = df_team_economy["source_url"].apply(
    match_id_vlr
)
df_team_economy["vlr_id-map"] = (
    df_team_economy["vlr_id"] + "-" + df_team_economy["map"]
)

df_team_economy = pd.merge(df_team_economy, df_maps_id[["vlr_id-map","map_id","series_id","reg_id","tour_id"]],how="left", left_on="vlr_id-map", right_on="vlr_id-map")

df_team_economy.to_csv(path_or_buf='tables/table_team_economy.csv',index=False,encoding='iso-8859-1')


In [90]:
#player stats

df_player_stats = concat_csv_from_different_folders(folder="csv", prefix="player_stats")

df_filter_map = df_player_stats["map"] != "all"
df_player_stats = df_player_stats[df_filter_map]
df_player_stats["vlr_id"] = df_player_stats["source_url"].apply(
    match_id_vlr
)
df_player_stats["vlr_id-map"] = (
    df_player_stats["vlr_id"] + "-" + df_player_stats["map"]
)

df_player_stats = pd.merge(df_player_stats, df_maps_id[["vlr_id-map","map_id","series_id","reg_id","tour_id"]],how="left", left_on="vlr_id-map", right_on="vlr_id-map")

df_player_stats.to_csv(path_or_buf='tables/table_player_stats.csv',index=False,encoding='iso-8859-1')


In [ ]:
#draft (raro hay que revisar)
df_draft_concat = concat_csv_from_different_folders(folder="csv", prefix="draft_")
df_draft_concat["match_instance"] = df_draft_concat["source_url"].apply(
    get_game_instance
)
df_draft_concat["vlr_id"] = df_draft_concat["source_url"].apply(match_id_vlr)

df_draft_concat = pd.merge(
    df_draft_concat,
    df_maps_id[["vlr_id", "series_id", "reg_id", "tour_id"]],
    how="left",
    left_on="vlr_id",
    right_on="vlr_id",
)
df_draft_concat.drop_duplicates(inplace=True, ignore_index=True)

df_draft_concat_fix = create_draft_table(df_draft_concat)

df_draft_concat_fix['1st_ban'] = df_draft_concat_fix.apply(first_ban,axis=1)

df_draft_concat_fix['2nd_ban'] = df_draft_concat_fix.apply(second_ban,axis=1)

df_draft_concat_fix['1st_pick'] = df_draft_concat_fix.apply(first_pick, axis=1)

df_draft_concat_fix['2nd_pick'] = df_draft_concat_fix.apply(second_pick,axis=1)

df_draft_concat_fix['decider'] = df_draft_concat_fix.apply(decider_pick,axis=1)

df_draft_concat_fix.to_csv(path_or_buf='tables/table_draft.csv',index=False,encoding='iso-8859-1')


In [135]:
# round stats

df_round_concat = concat_csv_from_different_folders(folder="csv", prefix="round_detail")

#table with the maps
df_maps_name_id = pd.DataFrame(df_round_concat["map"].unique(), columns=["map"])
text_to_index(df_maps_name_id,"map")

df_maps_name_id.to_csv(path_or_buf='tables/table_maps_name_id.csv',index=False,encoding='iso-8859-1')

df_round_concat["vlr_id"] = df_round_concat["source_url"].apply(
    match_id_vlr
)
df_round_concat["vlr_id-map"] = (
    df_round_concat["vlr_id"] + "-" + df_round_concat["map"]
)

df_round_concat = pd.merge(df_round_concat, df_maps_id[["vlr_id-map","map_id","series_id","reg_id","tour_id"]],how="left", left_on="vlr_id-map", right_on="vlr_id-map")

df_round_concat.to_csv(path_or_buf='tables/table_round_info.csv',index=False,encoding='iso-8859-1')


In [132]:
df_maps_id[df_maps_id["series_id"] == "series_95"]

,vlr_id-map,map_id,vlr_id,map,series_id,reg_id,tour_id
236,450076-Split,map_236,450076,Split,series_95,reg_2,tour_2
237,450076-Fracture,map_237,450076,Fracture,series_95,reg_2,tour_2


In [133]:
df_player_performance.describe()

,2K,3K,4K,5K,1v1,1v2,1v3,1v4,1v5,ECON,PL,DE
count,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000,3680.000000
mean,2.623641,0.788859,0.166033,0.019022,0.204348,0.111141,0.027174,0.003804,0.000272,51.713587,1.470652,0.437772
std,1.645188,0.921924,0.415634,0.136620,0.457587,0.330370,0.165921,0.061570,0.016485,16.289112,1.845694,0.690822
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,1.000000,0.000000
75%,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,61.000000,2.000000,1.000000
max,11.000000,5.000000,3.000000,1.000000,3.000000,3.000000,2.000000,1.000000,1.000000,180.000000,11.000000,5.000000


In [16]:
df_player_stats[df_player_stats["ratingT"] > 4]

,team,player,agent,ratingBoth,ratingT,rating-ct,acsBoth,acsT,acsCT,killsBoth,...,fdCT,fk-fdBoth,fk-fdT,fk-fdCT,map,date,event,date-map,map_id,series_id
1991,NAVI,koalanoob,Raze,1.75,4.07,1.56,359.0,683.0,333.0,16.0,...,0.0,5.0,1.0,1.0,Split,2025-04-09 13:15:00,VCT 2025: EMEA Stage 1,2025-04-09 13:15:00-Split,map_304,series_121
